In [14]:
import pandas as pd

df = pd.read_csv('data/final_nct_121_with_effecacy_group.csv')
df

,ticker,date_of_interest,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,...,start_date,has_primary_outcome,has_secondary_outcome,drug_routes,drug_classes,in_orange_drugs,all_drugs,find_in_orange,word,effecacy_group
0,ALPMY,"Aug 4, 2020",ENZALUTAMIDE,TABLET;ORAL,XTANDI,ASTELLAS,80MG,N,213674,2,...,2014-04-23,Yes,Yes,Oral,Chemotherapy,['ENZALUTAMIDE'],"['Enzalutamide', 'Placebo']",True,"ENZALUTAMIDE, 1, 0",more
1,SMPQY,"Jul 16, 2018",GEMCITABINE HYDROCHLORIDE,SOLUTION;INTRAVENOUS,INFUGEM,SUN PHARM,EQ 2200MG BASE/220ML (EQ 10MG BASE/ML),N,208313,10,...,2005-05,Yes,Yes,Intravenous,Antimetabolites|Platinum-based Chemotherapy,['GEMCITABINE HYDROCHLORIDE'],"['cisplatin', 'gemcitabine hydrochloride']",True,"GEMCITABINE HYDROCHLORIDE, 2, G",more
2,JNJ,"Feb 17, 2023",APALUTAMIDE,TABLET;ORAL,ERLEADA,JANSSEN BIOTECH,240MG,N,210951,2,...,2014-11-26,Yes,Yes,Oral,Corticosteroids,"['APALUTAMIDE', 'ABIRATERONE ACETATE', 'PREDNI...","['Apalutamide', 'Abiraterone acetate', 'Predni...",True,"APALUTAMIDE, 1, AP",more
3,NOT_LISTED,"May 22, 2023",PEMETREXED DISODIUM,SOLUTION;INTRAVENOUS,PEMETREXED,SHILPA,EQ 500MG/50ML BASE (10MG/ML),N,215179,2,...,2018-12-06,Yes,Yes,Intravenous,Pd-l1 Inhibitors|Antimetabolites|Platinum-base...,"['PEMETREXED', 'PACLITAXEL']","['Durvalumab', 'Placebo', 'Carboplatin', 'Cisp...",True,"PEMETREXED, 1, 1",more
4,ALPMY,"Aug 4, 2020",ENZALUTAMIDE,TABLET;ORAL,XTANDI,ASTELLAS,80MG,N,213674,2,...,2009-09-30,Yes,Yes,Oral,Chemotherapy|Taxanes|Monoclonal Antibodies,['ENZALUTAMIDE'],"['Enzalutamide', 'Placebo']",True,"ENZALUTAMIDE, 1, 0",more
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,NOT_LISTED,"Nov 20, 2023",CYCLOPHOSPHAMIDE,SOLUTION;INTRAVENOUS,CYCLOPHOSPHAMIDE,EUGIA PHARMA SPECLTS,2GM/10ML (200MG/ML),N,210735,3,...,2000-02,Yes,Yes,Intravenous|Oral,Corticosteroids|Monoclonal Antibodies,['CYCLOPHOSPHAMIDE'],"['Cyclophosphamide', 'Radiotherapy', 'Vincrist...",True,"CYCLOPHOSPHAMIDE, 1",more
117,ABBV,"Apr 11, 2016",VENETOCLAX,TABLET;ORAL,VENCLEXTA,ABBVIE,100MG,N,208573,3,...,2017-02-02,Yes,Yes,Intravenous|Oral|Subcutaneous,Chemotherapy|Monoclonal Antibodies,"['AZACITIDINE', 'VENETOCLAX']","['Azacitidine', 'Venetoclax', 'Placebo']",True,"VENETOCLAX, 1",more
118,NOT_LISTED,"Sep 14, 2016",ASPIRIN; OMEPRAZOLE,"TABLET, DELAYED RELEASE;ORAL",YOSPRALA,GENUS LIFESCIENCES,325MG;40MG,N,205103,2,...,2004-05,Yes,Yes,Oral,Unknown,['ASPIRIN'],['aspirin'],True,"ASPIRIN, 1",more
119,NOT_LISTED,"Oct 3, 2019",DEXAMETHASONE,TABLET;ORAL,HEMADY,DEXCEL,20MG,N,211379,1,...,1996-10,Yes,Yes,Unknown,Corticosteroids,['DEXAMETHASONE'],"['dexamethasone', 'doxorubicin hydrochloride',...",True,"DEXAMETHASONE, 2",more


In [20]:
df['ticker'].fillna('Not_Listed', inplace=True)
df['ticker'] = df['ticker'].str.upper()
df['ticker'] = [str(i).strip('\n') for i in df['ticker']]
df.to_csv('final_nct_121_with_effecacy_group.csv', index=False)

/var/folders/6b/fv26_4wd3_7cvw1h0zl_64v40000gn/T/ipykernel_9948/565850785.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ticker'].fillna('Not_Listed', inplace=True)


In [19]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm
from datetime import datetime, timedelta


# ── 2.  Helper that returns the first trading‑day prices on/after `date_str` ─
def get_open_close(sym: str, date_str: str):
    """
    Returns (open, close) for `sym` on the first trading day that is
    >= approval date.  If the ticker is missing or yfinance has no data,
    returns (None, None).
    """
    if not sym:                       # skip rows with no ticker (e.g., Dexamethasone)
        return 'Not_Listed', 'Not_Listed'
    if sym == 'Not_Listed':
        return 'Not_Listed', 'Not_Listed'

    # convert "Aug 4, 2020" → datetime.date
    target = datetime.strptime(date_str, "%b %d, %Y").date()

    # try up to 10 subsequent calendar days in case the date is a weekend/holiday
    for i in range(10):
        day = target + timedelta(days=i)
        # yfinance wants strings in ISO format
        start = day.isoformat()
        end   = (day + timedelta(days=1)).isoformat()

        try:
            hist = yf.Ticker(sym).history(start=start, end=end, auto_adjust=False)
            if not hist.empty:
                o = float(hist["Open"].iloc[0])
                c = float(hist["Close"].iloc[-1])
                return o, c
        except Exception:
            print(f"Failed to get data for {sym} on {date_str}")
    # give up if we never found data
    return 'Not_Listed', 'Not_Listed'

# ── 3.  Loop through the DataFrame and populate prices  ───────────────
opens, closes = [], []
for sym, dt in tqdm(zip(df["ticker"], df["date_of_interest"]), total=len(df)):
    o, c = get_open_close(sym, dt)
    opens.append(o)
    closes.append(c)

df["Open_on_Approval"]  = opens
df["Close_on_Approval"] = closes

# ── 4.  Inspect the result  ───────────────────────────────────────────
print(df)


  0%|          | 0/121 [00:00<?, ?it/s]

Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for ALPMY on Aug 4, 2020


  1%|          | 1/121 [00:10<20:04, 10.03s/it]

Failed to get data for ALPMY on Aug 4, 2020
Failed to get data for SMPQY on Jul 16, 2018
Failed to get data for SMPQY on Jul 16, 2018
Failed to get data for SMPQY on Jul 16, 2018
Failed to get data for SMPQY on Jul 16, 2018
Failed to get data for SMPQY on Jul 16, 2018
Failed to get data for SMPQY on Jul 16, 2018


  1%|          | 1/121 [00:18<37:45, 18.88s/it]


KeyboardInterrupt: 

In [9]:
df

,ticker,date_of_interest,Ingredient,DF;Route,Trade_Name,Applicant,Strength,Appl_Type,Appl_No,Product_No,...,has_secondary_outcome,drug_routes,drug_classes,in_orange_drugs,all_drugs,find_in_orange,word,effecacy_group,Open_on_Approval,Close_on_Approval
0,ALPMY,"Aug 4, 2020",ENZALUTAMIDE,TABLET;ORAL,XTANDI,ASTELLAS,80MG,N,213674,2,...,Yes,Oral,Chemotherapy,['ENZALUTAMIDE'],"['Enzalutamide', 'Placebo']",True,"ENZALUTAMIDE, 1, 0",more,Not_Listed,Not_Listed
1,SMPQY,"Jul 16, 2018",GEMCITABINE HYDROCHLORIDE,SOLUTION;INTRAVENOUS,INFUGEM,SUN PHARM,EQ 2200MG BASE/220ML (EQ 10MG BASE/ML),N,208313,10,...,Yes,Intravenous,Antimetabolites|Platinum-based Chemotherapy,['GEMCITABINE HYDROCHLORIDE'],"['cisplatin', 'gemcitabine hydrochloride']",True,"GEMCITABINE HYDROCHLORIDE, 2, G",more,Not_Listed,Not_Listed
2,JNJ,"Feb 17, 2023",APALUTAMIDE,TABLET;ORAL,ERLEADA,JANSSEN BIOTECH,240MG,N,210951,2,...,Yes,Oral,Corticosteroids,"['APALUTAMIDE', 'ABIRATERONE ACETATE', 'PREDNI...","['Apalutamide', 'Abiraterone acetate', 'Predni...",True,"APALUTAMIDE, 1, AP",more,Not_Listed,Not_Listed
3,NOT_LISTED,"May 22, 2023",PEMETREXED DISODIUM,SOLUTION;INTRAVENOUS,PEMETREXED,SHILPA,EQ 500MG/50ML BASE (10MG/ML),N,215179,2,...,Yes,Intravenous,Pd-l1 Inhibitors|Antimetabolites|Platinum-base...,"['PEMETREXED', 'PACLITAXEL']","['Durvalumab', 'Placebo', 'Carboplatin', 'Cisp...",True,"PEMETREXED, 1, 1",more,Not_Listed,Not_Listed
4,ALPMY,"Aug 4, 2020",ENZALUTAMIDE,TABLET;ORAL,XTANDI,ASTELLAS,80MG,N,213674,2,...,Yes,Oral,Chemotherapy|Taxanes|Monoclonal Antibodies,['ENZALUTAMIDE'],"['Enzalutamide', 'Placebo']",True,"ENZALUTAMIDE, 1, 0",more,Not_Listed,Not_Listed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,NOT_LISTED,"Nov 20, 2023",CYCLOPHOSPHAMIDE,SOLUTION;INTRAVENOUS,CYCLOPHOSPHAMIDE,EUGIA PHARMA SPECLTS,2GM/10ML (200MG/ML),N,210735,3,...,Yes,Intravenous|Oral,Corticosteroids|Monoclonal Antibodies,['CYCLOPHOSPHAMIDE'],"['Cyclophosphamide', 'Radiotherapy', 'Vincrist...",True,"CYCLOPHOSPHAMIDE, 1",more,Not_Listed,Not_Listed
117,ABBV,"Apr 11, 2016",VENETOCLAX,TABLET;ORAL,VENCLEXTA,ABBVIE,100MG,N,208573,3,...,Yes,Intravenous|Oral|Subcutaneous,Chemotherapy|Monoclonal Antibodies,"['AZACITIDINE', 'VENETOCLAX']","['Azacitidine', 'Venetoclax', 'Placebo']",True,"VENETOCLAX, 1",more,Not_Listed,Not_Listed
118,NOT_LISTED,"Sep 14, 2016",ASPIRIN; OMEPRAZOLE,"TABLET, DELAYED RELEASE;ORAL",YOSPRALA,GENUS LIFESCIENCES,325MG;40MG,N,205103,2,...,Yes,Oral,Unknown,['ASPIRIN'],['aspirin'],True,"ASPIRIN, 1",more,Not_Listed,Not_Listed
119,NOT_LISTED,"Oct 3, 2019",DEXAMETHASONE,TABLET;ORAL,HEMADY,DEXCEL,20MG,N,211379,1,...,Yes,Unknown,Corticosteroids,['DEXAMETHASONE'],"['dexamethasone', 'doxorubicin hydrochloride',...",True,"DEXAMETHASONE, 2",more,Not_Listed,Not_Listed
